In [1]:
import yfinance as yf
import pandas as pd
import sys
from pathlib import Path

root_path = str(Path.cwd().parent)
if root_path not in sys.path:
    sys.path.append(root_path)

from utils import preencher_nulos
from utils import formatar_eixo_ano

In [2]:
# 1. Parâmetros
ativo = "ITUB3.SA"
data_inicial = "2022-01-01"
data_final = "2025-12-31"

# 2. Download dos dados
tabela_cotacao = yf.download(
    ativo,
    start=data_inicial,
    end=data_final,
    progress=False,
    multi_level_index=False,
)

# 3. Cálculos de variação
tabela_cotacao["Variacao"] = tabela_cotacao["Close"].diff()
tabela_cotacao["Var%"] = tabela_cotacao["Close"].pct_change() * 100

# 4. Transformando o índice em datetime e garantindo que ele seja o Index
tabela_cotacao.index = pd.to_datetime(tabela_cotacao.index)
tabela_cotacao.columns.name = None

col_nulo = ["Variacao", "Var%"]

tabela_cotacao = preencher_nulos(tabela_cotacao, col_nulo, 0)

tabela_cotacao.head(10)

,Close,High,Low,Open,Volume,Variacao,Var%
Date,,,,,,,
2022-01-03,12.387838,12.579601,12.170507,12.330310,1461343,0.000000,0.000000
2022-01-04,12.662700,12.688267,12.368664,12.368664,1115778,0.274861,2.218800
2022-01-05,12.407016,12.726619,12.387839,12.643521,984350,-0.255684,-2.019189
2022-01-06,12.630734,12.656303,12.368660,12.598776,2113838,0.223719,1.803162
2022-01-07,12.828888,12.848064,12.438971,12.681870,1116684,0.198153,1.568820
2022-01-10,12.899209,12.931171,12.688271,12.828897,964862,0.070321,0.548146
2022-01-11,13.084573,13.135710,12.860850,12.892811,1354841,0.185364,1.437017
2022-01-12,13.090963,13.206020,12.918377,13.084571,1132206,0.006391,0.048841
2022-01-13,13.257160,13.397785,13.020653,13.039829,1303063,0.166197,1.269554


In [3]:
caminho_raiz = Path.cwd().parent

pasta_parquet = caminho_raiz / "arquivos_parquet"
pasta_parquet.mkdir(parents=True, exist_ok=True)

caminho_arquivo = pasta_parquet / "dados_base.parquet"
tabela_cotacao.to_parquet(caminho_arquivo)

print(f"Arquivo salvo em: {caminho_arquivo}")

Arquivo salvo em: c:\Users\Pc Gamer\OneDrive\FIAP\Machine Learnig Engineering\Fase - 04\fiap_tech_challenge_fase4\arquivos_parquet\dados_base.parquet
